# Qualitative analysis of outlier detection 

In [2]:
import pandas as pd
import json
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
import folium
import subprocess
from io import StringIO

In [3]:
def visualization(trajectory_df,color="blue"):    
    m2 = folium.Map(location=[(np.max(trajectory_df["latitude"])+np.min(trajectory_df["latitude"]))/2, (np.max(trajectory_df["longitude"])+np.min(trajectory_df["longitude"]))/2], zoom_start=14, width="70%", height="100") # create the map
    ids = trajectory_df["id"].unique() # get all uniques ids
    for i in ids: # add a line per trajectory and avoid linked trajectories
        tmp_df = trajectory_df[trajectory_df["id"] == i]
        coords = [zip(tmp_df["latitude"], tmp_df["longitude"])]
        folium.PolyLine(coords,
                        color=color,
                        weight=4,
                        opacity=0.7).add_to(m2)
    return m2

## Qualititative analysis based on the shapefile

### Trajectory loading

In [4]:
threshold = 3
df = pd.read_csv("datasets/my_traj.csv")
df['time'] = pd.to_datetime(df['time'])
df['time'] = df["time"].values.astype(np.int64)
df = df.sort_values(by=['id', 'time'])
df = df[["id","time","latitude","longitude"]]
df.to_csv(f"datasets/movetk_dataset/my_traj.csv",index=False)
df

,id,time,latitude,longitude
0,0,1688132053000000000,50.839978,4.365087
1,0,1688132055000000000,50.840059,4.365243
2,0,1688132057000000000,50.840115,4.365356
3,0,1688132095000000000,50.840184,4.365509
4,0,1688132098000000000,50.840269,4.365628
...,...,...,...,...
312,0,1688133335000000000,50.821950,4.404838
313,0,1688133337000000000,50.821916,4.405050
314,0,1688133339000000000,50.821854,4.405199
315,0,1688133341000000000,50.821825,4.405334


### Shapefile loading & Ground truth computation

In [5]:
from pyproj import CRS, Geod
line = "034b"
variante = 1
shapefile= pd.read_csv("datasets/shapefiles.csv", delimiter=";")
shapefile = shapefile[(shapefile["LIGNE"]==line) & (shapefile["VARIANTE"]==variante )]["Geo Shape"].values[0]
shapefile = json.loads(shapefile)

crs_utm = CRS.from_user_input(31370)
geod = crs_utm.get_geod()  # Your data may be from a different Geod.
distance = np.zeros(len(df))
nearest_points_list = []
df = df.reset_index(drop=True)
line_shape = LineString(shapefile["coordinates"]) 
for index, row in df.iterrows():
    point = Point(row["longitude"], row["latitude"])
    closest_points = nearest_points(line_shape, point)
    nearest_points_list.append(closest_points[0])
    distance[index] = geod.geometry_length(LineString(closest_points))
    
df["distance"] = distance 
#df["nearest_point"] = nearest_points_list
mean = df["distance"].mean()
median = df["distance"].median()
std = df["distance"].std()
k=1.5
threshold = median + k * std
#df["nearest_point"] = nearest_points_list
print(mean, std)
outliers = df[df["distance"] > threshold]
is_normal = np.zeros(len(df))
for i in range(len(df['time'].values)):
    if df['time'].values[i] not in outliers['time'].values:
        is_normal[i] = 1
df["is normal"] = is_normal > 0

df.to_csv(f"datasets/movetk_dataset/my_traj.csv",index=False)
df

6.99931376823816 6.984997264301602


,id,time,latitude,longitude,distance,is normal
0,0,1688132053000000000,50.839978,4.365087,6.434879,True
1,0,1688132055000000000,50.840059,4.365243,6.241407,True
2,0,1688132057000000000,50.840115,4.365356,6.274832,True
3,0,1688132095000000000,50.840184,4.365509,6.957367,True
4,0,1688132098000000000,50.840269,4.365628,4.605384,True
...,...,...,...,...,...,...
312,0,1688133335000000000,50.821950,4.404838,4.315869,True
313,0,1688133337000000000,50.821916,4.405050,1.384736,True
314,0,1688133339000000000,50.821854,4.405199,3.199198,True
315,0,1688133341000000000,50.821825,4.405334,2.081433,True


#### Optimal speed bounded method

In [6]:
a = subprocess.run(["/home/pierre-cedric/Documents/MEMO-F524/movetk/build/examples/outlier", f"/home/pierre-cedric/Documents/MEMO-F524/datasets/movetk_dataset/my_traj.csv", str(9)], capture_output=True)
txt = StringIO(a.stdout.decode("utf-8"))
cleaned_df = pd.read_csv(txt, sep=",")
cleaned_df.to_csv('csv_results/cleaned_df_trip.csv', index=False)
cleaned_df = cleaned_df.rename(columns={"Longitude": "longitude", "Latitude": "latitude", "Timestamp": "time"})
#cleaned_df["time"] = cleaned_df["Timestamp"]
cleaned_df = cleaned_df.merge(df.drop_duplicates(), on=['time'], how='left', indicator=True)[["id_x","time","latitude_x","longitude_x","is normal"]]
cleaned_df

,id_x,time,latitude_x,longitude_x,is normal
0,0,1688132053000000000,50.839978,4.365087,True
1,0,1688132057000000000,50.840115,4.365356,True
2,0,1688132095000000000,50.840184,4.365509,True
3,0,1688132098000000000,50.840269,4.365628,True
4,0,1688132100000000000,50.840344,4.365725,True
...,...,...,...,...,...
136,0,1688133250000000000,50.824581,4.399835,True
137,0,1688133337000000000,50.821916,4.405050,True
138,0,1688133339000000000,50.821854,4.405199,True
139,0,1688133341000000000,50.821825,4.405334,True


In [25]:
outlier_df = df.merge(cleaned_df.drop_duplicates(), on=['time'], how='left', indicator=True)
outlier_df = outlier_df[outlier_df["_merge"] == "left_only"]
outlier_df

,id,time,latitude,longitude,distance,is normal_x,id_x,latitude_x,longitude_x,is normal_y,_merge
1,0,1688132055000000000,50.840059,4.365243,6.241407,True,NaN,NaN,NaN,NaN,left_only
16,0,1688132167000000000,50.840175,4.367074,17.310377,False,NaN,NaN,NaN,NaN,left_only
17,0,1688132170000000000,50.840135,4.367321,15.817336,False,NaN,NaN,NaN,NaN,left_only
18,0,1688132172000000000,50.840150,4.367540,8.997086,True,NaN,NaN,NaN,NaN,left_only
19,0,1688132174000000000,50.840063,4.367721,14.449077,True,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
308,0,1688133327000000000,50.822242,4.404106,0.925815,True,NaN,NaN,NaN,NaN,left_only
309,0,1688133329000000000,50.822194,4.404319,3.525084,True,NaN,NaN,NaN,NaN,left_only
310,0,1688133331000000000,50.822144,4.404516,5.469421,True,NaN,NaN,NaN,NaN,left_only
311,0,1688133333000000000,50.822050,4.404675,1.126434,True,NaN,NaN,NaN,NaN,left_only


In [26]:
TP = len(cleaned_df[cleaned_df["is normal"] == True])
FP = len(cleaned_df[cleaned_df["is normal"] == False])
TN = len(outlier_df[outlier_df["is normal_x"] == False])
FN = len(outlier_df[outlier_df["is normal_x"] == True])
P = TP / (TP + FP)
A = (TP + TN) / (TP + TN + FP + FN)
R = TP / (TP + FN)
F1 = 2 * (P * R) / (P + R)
print("TP:{}\nFP:{}\nTN:{}\nFN:{}".format(TP,FP,TN,FN))
print("Precision:", round(P,3))
print("Accuracy::",round(A,3))
print("Recall:", round(R,3))
print("F1:", round(F1,3))

TP:122
FP:19
TN:16
FN:160
Precision: 0.865
Accuracy:: 0.435
Recall: 0.433
F1: 0.577


## Qualititative analysis based on the introduction of outliers

### Ground truth computation

In [9]:
PROB = 0.99
EPS = 10**(-3)
FILE = "cars_8.csv"
df= pd.read_csv(f"datasets/{FILE}")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Timestamp'] = df["Timestamp"].values.astype(np.int64)
df = df.sort_values(by=['id', 'Timestamp'])
df = df[["id","Timestamp","Latitude","Longitude"]]
variation_lat = np.zeros(len(df))
variation_lon = np.zeros(len(df))
is_normal = np.zeros(len(df))
for i in range(len(variation_lat)):
    p = np.random.random()
    if p > PROB:
        variation_lat[i] = np.random.random()*EPS
    p = np.random.random()
    if p > PROB:
        variation_lon[i] = np.random.random()*EPS
    if variation_lat[i] == 0 and variation_lon[i] == 0:
        is_normal[i] = 1
df["Latitude"] += variation_lat
df["Longitude"] += variation_lon
df["is normal"] = is_normal > 0
df[df["is normal"] == 0 ]
df = df.rename(columns={"Longitude": "longitude", "Latitude": "latitude", "Timestamp": "time"})
df.to_csv(f"datasets/movetk_dataset/{FILE}",index=False)
df

,id,time,latitude,longitude,is normal
0,8,1591001768148000000,50.861073,4.465373,True
1,8,1591001769648000000,50.861100,4.466186,False
2,8,1591001770398000000,50.861127,4.465398,True
3,8,1591001770797745000,50.861146,4.465406,True
4,8,1591001784229685000,50.861756,4.465684,True
...,...,...,...,...,...
4937,8,1591307819514958000,50.861842,4.465724,True
4938,8,1591307819943340000,50.861822,4.465716,True
4939,8,1591307821415195000,50.861756,4.465684,True
4940,8,1591307834847135000,50.861146,4.465406,True


In [7]:
df= pd.read_csv(f"datasets/berlinMOD_sample_.csv")
df['time'] = pd.to_datetime(df['time'])
df['time'] = df["time"].values.astype(np.int64)
df = df.sort_values(by=['id', 'time'])
df = df[["id","time","latitude","longitude","is normal"]]
df.to_csv(f"datasets/movetk_dataset/berlinMOD_sample_.csv",index=False)
df

,id,time,latitude,longitude,is normal
0,8,1591001768148000000,50.861073,4.465373,True
1,8,1591001769648000000,50.861100,4.465386,True
2,8,1591001770398000000,50.861127,4.465398,True
3,8,1591001770797745000,50.861146,4.465406,True
4,8,1591001784229685000,50.861756,4.465684,True
...,...,...,...,...,...
4937,8,1591307819514958000,50.861842,4.465724,True
4938,8,1591307819943340000,50.861822,4.465716,True
4939,8,1591307821415195000,50.861756,4.465684,True
4940,8,1591307834847135000,50.861146,4.465406,True


#### Optimal speed bounded method

In [8]:
a = subprocess.run(["/home/pierre-cedric/Documents/MEMO-F524/movetk/build/examples/outlier", f"/home/pierre-cedric/Documents/MEMO-F524/datasets/movetk_dataset/berlinMOD_sample_.csv", "25"], capture_output=True)
txt = StringIO(a.stdout.decode("utf-8"))
cleaned_df = pd.read_csv(txt, sep=",")
cleaned_df.to_csv('csv_results/cleaned_df_MOD.csv', index=False)
#cleaned_df["time"] = cleaned_df["Timestamp"]
cleaned_df = cleaned_df.merge(df.drop_duplicates(), on=['time'], how='left', indicator=True)
cleaned_df

,id_x,time,latitude_x,longitude_x,id_y,latitude_y,longitude_y,is normal,_merge
0,8,1591001768148000000,50.861073,4.465373,8,50.861073,4.465373,True,both
1,8,1591001769648000000,50.861100,4.465386,8,50.861100,4.465386,True,both
2,8,1591001770398000000,50.861127,4.465398,8,50.861127,4.465398,True,both
3,8,1591001770797745000,50.861146,4.465406,8,50.861146,4.465406,True,both
4,8,1591001784229685000,50.861756,4.465684,8,50.861756,4.465684,True,both
...,...,...,...,...,...,...,...,...,...
4838,8,1591307819514958000,50.861842,4.465724,8,50.861842,4.465724,True,both
4839,8,1591307819943340000,50.861822,4.465716,8,50.861822,4.465716,True,both
4840,8,1591307821415195000,50.861756,4.465684,8,50.861756,4.465684,True,both
4841,8,1591307834847135000,50.861146,4.465406,8,50.861146,4.465406,True,both


In [42]:
outlier_df = df.merge(cleaned_df.drop_duplicates()[["id_x","time","latitude_x","longitude_x","is normal"]], on=['time'], how='left', indicator=True)
outlier_df = outlier_df[outlier_df["_merge"] == "left_only"]
outlier_df

,id,time,latitude,longitude,is normal_x,id_x,latitude_x,longitude_x,is normal_y,_merge
11,8,1591001788127688000,50.861891,4.520971,False,NaN,NaN,NaN,NaN,left_only
81,8,1591001880120432000,50.918525,4.463031,False,NaN,NaN,NaN,NaN,left_only
122,8,1591001928919558000,50.864420,4.478834,False,NaN,NaN,NaN,NaN,left_only
189,8,1591001983575451000,50.864706,4.483312,False,NaN,NaN,NaN,NaN,left_only
343,8,1591002136763272000,50.869571,4.549198,False,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
4758,8,1591303808317145000,50.879922,4.463085,False,NaN,NaN,NaN,NaN,left_only
4780,8,1591303830665938000,50.882729,4.462464,False,NaN,NaN,NaN,NaN,left_only
4822,8,1591307688628619000,50.864398,4.511244,False,NaN,NaN,NaN,NaN,left_only
4858,8,1591307729189371000,50.863388,4.496401,False,NaN,NaN,NaN,NaN,left_only


In [43]:
TP = len(cleaned_df[cleaned_df["is normal"] == True])
FP = len(cleaned_df[cleaned_df["is normal"] == False])
TN = len(outlier_df[outlier_df["is normal_x"] == False])
FN = len(outlier_df[outlier_df["is normal_x"] == True])
P = TP / (TP + FP)
A = (TP + TN) / (TP + TN + FP + FN)
R = TP / (TP + FN)
F1 = 2 * (P * R) / (P + R)
print("TP:{}\nFP:{}\nTN:{}\nFN:{}".format(TP,FP,TN,FN))
print("Precision:", round(P,3))
print("Accuracy::",round(A,3))
print("Recall:", round(R,3))
print("F1:", round(F1,3))

TP:4843
FP:0
TN:99
FN:0
Precision: 1.0
Accuracy:: 1.0
Recall: 1.0
F1: 1.0
